  Using cached Keras_Applications-1.0.8-py3-none-any.whl (50 kB)
ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL


In [6]:
from imutils.video import VideoStream
import cv2
import imutils
import os
import time
import numpy as np
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from keras_vggface.utils import decode_predictions
import PIL
import numpy as np


In [7]:
detectorPaths = {
	"face": "haarcascade_frontalface_default.xml",
	"eyes": "haarcascade_eye.xml",
	"smile": "haarcascade_smile.xml",
}

detector = {}

for (name,path) in detectorPaths.items():
    path = os.path.join("./haarcascades/",path)
    detector[name] = cv2.CascadeClassifier(path)

print(detector)

{'face': <CascadeClassifier 0x7ffbebd048d0>, 'eyes': <CascadeClassifier 0x7ffbebd04510>, 'smile': <CascadeClassifier 0x7ffbebd04590>}


In [8]:
#start video stream
model = VGGFace(model= 'resnet50')
# detector = MTCNN()
target_size = (224,224) # output image size
border_rel = 0 # increase or decrease zoom on image



In [9]:
vs = VideoStream(src=0).start()
time.sleep(2.0)
cv2.startWindowThread()

while True:
    
    frame = vs.read()
    frame = imutils.resize(frame,width=500)
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    face_rect = detector["face"].detectMultiScale(gray,scaleFactor=1.05, minNeighbors=5, minSize=(30, 30),
                                                  flags=cv2.CASCADE_SCALE_IMAGE)
    for (fX, fY, fW, fH) in face_rect:
        
        cv2.rectangle(gray,(fX, fY), (fX + fW, fY + fH),
                      (0, 255, 0), 2)
        
        x1, y1, width, height = fX, fY, fW, fH
        dw = round(width * border_rel)
        dh = round(height * border_rel)

        x2, y2 = x1 + width + dw, y1 + height + dh
#         face = frame[y1:y2, x1:x2]
        
#         faces = frame[fY : fY + fH , fX:fX + fW]
#         cv2.imshow("faces",faces)
    
    
        
        face = frame[y1:y2, x1:x2]
        face = PIL.Image.fromarray(face)
        face = face.resize((224, 224))
        face = np.asarray(face)

 
        face_pp = face.astype('float32')
        face_pp = np.expand_dims(face_pp, axis = 0)
        face_pp = preprocess_input(face_pp, version = 2)
    
        prediction = model.predict(face_pp)
        results = decode_predictions(prediction)
        
        # Display results
        cv2.imshow("face",face)
        
        if(results[0][0][1] * 100 > 90):
            print("found this")
            ans = input()
            play_video("play")
            vs.stop()
            for result in results[0]:
                print ('%s: %.3f%%' % (result[0], result[1]*100))
    
    cv2.imshow("Frame", gray)
    
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

cv2.destroyAllWindows()
vs.stop()


In [8]:
   #     face_rect = detector["face"].detectMultiScale(gray,scaleFactor=1.05, minNeighbors=5, minSize=(30, 30),
    #                                                   flags=cv2.CASCADE_SCALE_IMAGE)
    #     for (fX, fY, fW, fH) in face_rect:
    #         cv2.rectangle(gray,(fX, fY), (fX + fW, fY + fH),
    #                       (0, 255, 0), 2)

    #         faces = frame[fY : fY + fH , fX:fX + fW]
    #         cv2.imshow("face",faces)
    
    

0.001094596
